In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import log_loss

In [3]:
X_train = pd.read_csv('D:\\desktop files\\weka\\Datasets\\covid_symptoms_checker\\Cleaned-Data.csv')
y_train = X_train.pop('Severity_Severe').to_frame()
x_train = X_train.iloc[:,:25]

In [11]:
X_train

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,...,Gender_Female,Gender_Male,Gender_Transgender,Severity_Mild,Severity_Moderate,Severity_None,Contact_Dont-Know,Contact_No,Contact_Yes,Country
125151,1,0,0,0,0,0,1,1,1,0,...,0,0,1,0,1,0,0,0,1,Republic of Korean
36050,1,1,1,1,0,0,1,1,1,0,...,0,0,1,1,0,0,1,0,0,Italy
222457,0,1,1,1,1,0,1,0,0,0,...,0,1,0,1,0,0,0,1,0,UAE
250914,0,0,0,1,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,UAE
296710,0,1,1,0,0,0,0,0,1,0,...,0,0,1,0,0,1,0,1,0,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,0,0,0,1,1,0,1,1,1,0,...,0,0,1,0,0,1,1,0,0,Republic of Korean
259178,0,0,1,0,0,0,0,1,1,0,...,0,0,1,1,0,0,1,0,0,Other-EUR
131932,0,1,1,0,0,0,0,1,1,0,...,0,0,1,0,1,0,0,1,0,France
146867,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,1,1,0,0,France


In [4]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train,test_size=0.2,random_state=42)
X_val, X_test, y_val, y_test = train_test_split(
    X_val, y_val,test_size=0.5,random_state=42)

In [5]:
y_val.head()

,Severity_Severe
95410,0
30706,0
65925,0
238465,0
243080,1


In [6]:
model_cols = ['Fever','Tiredness','Dry-Cough',
              'Difficulty-in-Breathing',
              'Sore-Throat','None_Sympton',
              'Pains','Nasal-Congestion',
              'Runny-Nose','Diarrhea',
              'None_Experiencing','Age_0-9',
              'Age_10-19','Age_20-24','Age_25-59',
              'Age_60_','Gender_Female','Gender_Male',
              'Gender_Transgender','Contact_Dont-Know',
              'Contact_No','Contact_Yes']

In [7]:
Y_train_df = X_train[model_cols].copy()
Y_train_df.columns = Y_train_df.columns + "_out"
Y_train_df['Severity_Severe_out'] = y_train['Severity_Severe']
Y_train_df['Severity_Severe_aux_out'] = y_train['Severity_Severe']
trainset = tf.data.Dataset.from_tensor_slices((
    dict(X_train),dict(Y_train_df))).batch(256)
Y_val_df = X_val[model_cols].copy()
Y_val_df.columns = Y_val_df.columns + "_out"
Y_val_df['Severity_Severe_out'] = y_val['Severity_Severe']
Y_val_df['Severity_Severe_aux_out'] = y_val['Severity_Severe']
valset = tf.data.Dataset.from_tensor_slices((
    dict(X_val),dict(Y_val_df))).batch(256)
Y_test_df = X_test[model_cols].copy()
Y_test_df.columns = Y_test_df.columns + "_out"
Y_test_df['Severity_Severe_out'] = y_test['Severity_Severe']
Y_val_df['Severity_Severe_aux_out'] = y_val['Severity_Severe']
testset = tf.data.Dataset.from_tensor_slices((
    dict(X_test),dict(Y_test_df))).batch(256)

In [17]:
Y_test_df.head(2)

,Fever_out,Tiredness_out,Dry-Cough_out,Difficulty-in-Breathing_out,Sore-Throat_out,None_Sympton_out,Pains_out,Nasal-Congestion_out,Runny-Nose_out,Diarrhea_out,...,Age_20-24_out,Age_25-59_out,Age_60__out,Gender_Female_out,Gender_Male_out,Gender_Transgender_out,Contact_Dont-Know_out,Contact_No_out,Contact_Yes_out,Severity_Severe_out
124100,0,0,0,1,1,0,1,1,1,0,...,0,0,1,1,0,0,1,0,0,1
180486,0,1,1,0,0,0,1,0,0,0,...,0,1,0,1,0,0,0,0,1,1


In [8]:
def add_model(
    feature_outputs=None,hidden_layers=[[512,0],[64,0]],
    batch_norm=False,model_name=None,activation='sigmoid'):
    
    if batch_norm == True:
        layer = layers.BatchNormalization()(feature_outputs)
    else:
        layer = feature_outputs
    
    for i in range(len(hidden_layers)):
        layer = layers.Dense(hidden_layers[i][0], activation='relu',
                             name=model_name+'_L'+str(i))(layer)
        last_layer = layer
        
        if batch_norm == True:
            layer = layers.BatchNormalization()(layer)
        if hidden_layers[i][1] > 0:
            layer = layers.Dropout(hidden_layers[i][1])(layer)
        
    output_layer = layers.Dense(1, activation=activation,
                                name=model_name+'_out')(layer)
        
    return last_layer, output_layer

In [18]:
print(last_layer)

NameError: name 'last_layer' is not defined

In [9]:
emb_layers = {'Country':list(X_train['Country'].unique())}

In [10]:
def add_emb(emb_layers={},model_inputs={}):
    emb_inputs = {}
    emb_features = []
    
    for key,value in emb_layers.items():
        emb_inputs[key] = model_inputs[key]
        catg_col = feature_column.categorical_column_with_vocabulary_list(key, value)
        emb_col = feature_column.embedding_column(catg_col,dimension=int(len(value)**0.25))
        emb_features.append(emb_col)
    
    emb_layer = layers.DenseFeatures(emb_features)
    emb_outputs = emb_layer(emb_inputs)
    
    return emb_outputs

In [11]:
feature_layers = {col:[col,'Country'] for col in model_cols}
feature_layers['Gender_Female'] += ['Gender_Male',
                                    'Gender_Transgender']
feature_layers['Gender_Male'] += ['Gender_Female',
                                  'Gender_Transgender']
feature_layers['Gender_Transgender'] += ['Gender_Female',
                                         'Gender_Male']
feature_layers['Age_0-9'] += ['Age_10-19','Age_20-24',
                              'Age_25-59','Age_60_']
feature_layers['Age_10-19'] += ['Age_0-9','Age_20-24',
                                'Age_25-59','Age_60_']
feature_layers['Age_20-24'] += ['Age_0-9','Age_10-19',
                                'Age_25-59','Age_60_']
feature_layers['Age_25-59'] += ['Age_0-9','Age_10-19',
                                'Age_20-24','Age_60_']
feature_layers['Age_60_'] += ['Age_0-9','Age_10-19',
                              'Age_20-24','Age_25-59']
feature_layers['Contact_Dont-Know'] += ['Contact_No','Contact_Yes']
feature_layers['Contact_No'] += ['Contact_Dont-Know','Contact_Yes']
feature_layers['Contact_Yes'] += ['Contact_Dont-Know','Contact_No']

In [12]:
feature_layers['Severity_Severe_aux'] = ['Country']

In [13]:
def feature_models(
    output_feature=None,all_features=[],feature_layers={},
    emb_layers={},hidden_layers=[],batch_norm=False):
    
    model_inputs = {}
    for feature in all_features:
        if feature in [k for k,v in emb_layers.items()]:
            model_inputs[feature] = tf.keras.Input(shape=(1,),
                                                   name=feature,
                                                   dtype='string')
        else:
            model_inputs[feature] = tf.keras.Input(shape=(1,),
                                                   name=feature)
            
    if len(emb_layers) > 0:
        emb_outputs = add_emb(emb_layers,model_inputs)
        
    output_layers = []
    eng_layers = []
    for key,value in feature_layers.items():
        feature_columns = [feature_column.numeric_column(f)
                           for f in all_features if f not in value]
        
        feature_layer = layers.DenseFeatures(feature_columns)
        feature_outputs = feature_layer({k:v for k,v in
                                         model_inputs.items()
                                         if k not in value})
        
        if len(emb_layers) > 0:
            feature_outputs = layers.concatenate([feature_outputs,
                                                  emb_outputs])
    
        last_layer, output_layer = add_model(
            feature_outputs=feature_outputs,
            hidden_layers=hidden_layers,
            batch_norm=batch_norm,
            model_name=key)
        
        output_layers.append(output_layer)
        eng_layers.append(last_layer)
        
        if key == output_feature + '_aux':
            eng_layers.append(feature_outputs)
    
    return model_inputs, output_layers, eng_layers

In [14]:
params = {'all_features': list(X_train.columns),
          'output_feature':y_train.columns[0],
          'emb_layers':emb_layers,
          'feature_layers':feature_layers,
          'hidden_layers':[[256,0],[128,0.1],[64,0.2]],
          'batch_norm': True,
          'learning_rate':0.001,
          'patience':3,
          'epochs':20
        }

In [15]:
def final_model(params,test=True):
    print(params['batch_norm'],params['hidden_layers'])
    
    model_inputs, output_layers, eng_layers = feature_models(
        all_features=params['all_features'],
        feature_layers=params['feature_layers'],
        emb_layers=params['emb_layers'],
        hidden_layers=params['hidden_layers'],
        batch_norm=params['batch_norm'],
        output_feature=params['output_feature'])
    
    concat_layer = layers.concatenate(eng_layers)
    last_layer, output_layer = add_model(
        feature_outputs=concat_layer,
        hidden_layers=params['hidden_layers'],
        batch_norm=params['batch_norm'],
        model_name=params['output_feature'])
    
    output_layers.append(output_layer)
    
    model = tf.keras.Model(
        inputs=[model_inputs],
        outputs=output_layers)
        
    aux_loss_wgt = 0.5 / len(params['feature_layers'])
    loss_wgts = [aux_loss_wgt for i in range(len(params['feature_layers']))]
    loss_wgts.append(0.5)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(
                      lr=params["learning_rate"]),
                  loss_weights=loss_wgts,
                  metrics=['accuracy'])
    
    es = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',mode='min',verbose=1,
        patience=params['patience'],restore_best_weights=True)
                 
    history = model.fit(
        trainset,validation_data=valset,
        epochs=params['epochs'], verbose=0, callbacks=[es])
    
    yhat = model.predict(testset)
    loss = log_loss(
        np.array(y_test[params['output_feature']]),
        yhat[-1])**.5
    
    print('Binary Crossentropy:',loss)
    
    if test==True:
        sys.stdout.flush()
        return {'loss': loss, 'status': STATUS_OK}
    else:
        return history, model

In [16]:
 model_inputs, output_layers, eng_layers = feature_models(
        all_features=params['all_features'],
        feature_layers=params['feature_layers'],
        emb_layers=params['emb_layers'],
        hidden_layers=params['hidden_layers'],
        batch_norm=params['batch_norm'],
        output_feature=params['output_feature'])
    
concat_layer = layers.concatenate(eng_layers)
last_layer, output_layer = add_model(
    feature_outputs=concat_layer,
    hidden_layers=params['hidden_layers'],
    batch_norm=params['batch_norm'],
    model_name=params['output_feature'])

output_layers.append(output_layer)

model = tf.keras.Model(
    inputs=[model_inputs],
    outputs=output_layers)

aux_loss_wgt = 0.5 / len(params['feature_layers'])
loss_wgts = [aux_loss_wgt for i in range(len(params['feature_layers']))]
loss_wgts.append(0.5)

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(
                  lr=params["learning_rate"]),
              loss_weights=loss_wgts,
              metrics=['accuracy'])

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',mode='min',verbose=1,
    patience=params['patience'],restore_best_weights=True)

history = model.fit(
    trainset,validation_data=valset,
    epochs=params['epochs'], verbose=0, callbacks=[es])

yhat = model.predict(testset)
loss = log_loss(
    np.array(y_test[params['output_feature']]),
    yhat[-1])**.5

print('Binary Crossentropy:',loss)

if test==True:
    sys.stdout.flush()
    return {'loss': loss, 'status': STATUS_OK}
else:
    return history, model

Restoring model weights from the end of the best epoch.
Epoch 00011: early stopping


ValueError: in converted code:

    C:\Program Data\envs\Tensorflow\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py:677 map_fn
        batch_size=None)
    C:\Program Data\envs\Tensorflow\lib\site-packages\tensorflow_core\python\keras\engine\training.py:2469 _standardize_tensors
        exception_prefix='target')
    C:\Program Data\envs\Tensorflow\lib\site-packages\tensorflow_core\python\keras\engine\training_utils.py:510 standardize_input_data
        'for each key in: ' + str(names))

    ValueError: No data provided for "Severity_Severe_aux_out". Need data for each key in: ['Fever_out', 'Tiredness_out', 'Dry-Cough_out', 'Difficulty-in-Breathing_out', 'Sore-Throat_out', 'None_Sympton_out', 'Pains_out', 'Nasal-Congestion_out', 'Runny-Nose_out', 'Diarrhea_out', 'None_Experiencing_out', 'Age_0-9_out', 'Age_10-19_out', 'Age_20-24_out', 'Age_25-59_out', 'Age_60__out', 'Gender_Female_out', 'Gender_Male_out', 'Gender_Transgender_out', 'Contact_Dont-Know_out', 'Contact_

In [ ]:
ok, uk = add_model(feature_outputs=None,hidden_layers=[[512,0],[64,0]],batch_norm=False,model_name=None,activation='sigmoid'):

In [32]:
history, model = final_model(params,test=True)

True [[256, 0], [128, 0.1], [64, 0.2]]
Restoring model weights from the end of the best epoch.
Epoch 00013: early stopping


ValueError: in converted code:

    C:\Program Data\envs\Tensorflow\lib\site-packages\tensorflow_core\python\keras\engine\training_v2.py:677 map_fn
        batch_size=None)
    C:\Program Data\envs\Tensorflow\lib\site-packages\tensorflow_core\python\keras\engine\training.py:2469 _standardize_tensors
        exception_prefix='target')
    C:\Program Data\envs\Tensorflow\lib\site-packages\tensorflow_core\python\keras\engine\training_utils.py:510 standardize_input_data
        'for each key in: ' + str(names))

    ValueError: No data provided for "Severity_Severe_aux_out". Need data for each key in: ['Fever_out', 'Tiredness_out', 'Dry-Cough_out', 'Difficulty-in-Breathing_out', 'Sore-Throat_out', 'None_Sympton_out', 'Pains_out', 'Nasal-Congestion_out', 'Runny-Nose_out', 'Diarrhea_out', 'None_Experiencing_out', 'Age_0-9_out', 'Age_10-19_out', 'Age_20-24_out', 'Age_25-59_out', 'Age_60__out', 'Gender_Female_out', 'Gender_Male_out', 'Gender_Transgender_out', 'Contact_Dont-Know_out', 'Contact_